In [1]:
# labels_r, Reviewed=False,extra_tags
import glob
import csv

from pathlib import Path

def read_csv(csv_file_path, fieldnames=None):
    with open(csv_file_path, encoding='utf-8') as csv_file:
        rows = csv.DictReader(csv_file,fieldnames=fieldnames)
        rows = list(rows)
        for row in rows:
            for key,value in row.items():
                if value.lower() == 'false':
                    row[key]='FALSE'

    return rows

def write_csv(new_csv_file, rows_new,fieldnames=None):
    with open(new_csv_file, 'w', newline='',encoding='utf-8') as csvfile:
        if fieldnames is None:
            fieldnames = rows_new[0].keys()
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

        writer.writeheader()
        writer.writerows(rows_new)
        



In [2]:
file_name='/home/enis/projects/nna/notebooks/Labeling/datasetmodels_V6.csv'
new_file_name='/home/enis/projects/nna/notebooks/Labeling/results_megan/datasetmodels_V6_UI.csv'
dataset_rows=read_csv(file_name)


In [3]:
tag_names=list(dataset_rows[0].keys())[9:-3]
first_cols=list(dataset_rows[0].keys())[:9]
last_cols=list(dataset_rows[0].keys())[-3:]

In [4]:
# tag_names

In [6]:
for row in dataset_rows:
    for tag in tag_names:
        row[tag+'_r']='N/A'
    row['Reviewed'] = False
    row['extra_tags']=''

In [7]:
write_csv(new_file_name,dataset_rows)

In [8]:
# rows=read_csv('/home/enis/projects/nna/notebooks/explore/datasetUI_V5_short2k.csv')

In [9]:
# import shutil
# from pathlib import Path
# for row in rows:
#     dst=row['Clip Path']
#     Path(dst).parent.mkdir(parents=True,exist_ok=True)
#     src=row['Clip Path'].replace('yfitloiq-V1','yfitloiq-V1_backup')
#     shutil.copyfile(src, dst)


In [10]:
import librosa
import librosa.display
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path


In [11]:
row['Clip Path']

'/scratch/enis/data/nna/labeling/megan/combined7-V1/prudhoe/26/S4A10255_20190904_201602_67m_10s__67m_20s.wav'

In [ ]:
# /scratch/enis/data/nna/labeling/megan/yfitloiq-V1_mel/

for row in dataset_rows:
    fig, ax = plt.subplots(figsize=(32, 8))

    # print(c)
    cp=row['Clip Path']
    mel_path = cp.replace('combined7-V1','combined7-V1_mel')
    clip_path = Path(mel_path)
    clip_path.parent.mkdir(parents=True,exist_ok=True)
    c=clip_path.with_suffix('.svg')

    if c.exists():
        plt.close()
        continue

    y, sr = librosa.load(cp)

    S = librosa.feature.melspectrogram(y, sr=sr)
    S_DB = librosa.power_to_db(S, ref=np.max)

    
    img = librosa.display.specshow(S_DB,
                                   sr=sr,
                                   x_axis='time',
                                   y_axis='mel',
                                   ax=ax)
    fig.colorbar(img, format='%+2.0f dB', ax=ax)
    # plt.show()
    # break
    plt.savefig(c,bbox_inches='tight')
    plt.close()
    print(c)

    
    

In [13]:
!/scratch/enis/conda/envs/speechEnv/bin/python /home/enis/projects/nna/src/nna/slack_message.py -t "mel job ended" -m 'mel audio ended'


In [14]:
!rclone copy --s3-acl=public-read /scratch/enis/data/nna/labeling/megan/combined7-V1_mel/ enisdo:crescent/scratch/enis/data/nna/labeling/megan/combined7-V1_mel/

https://crescent.sfo3.digitaloceanspaces.com/scratch/enis/data/nna/labeling/megan/combined7-V1/anwr/34/S4A10327_20190714_053000_48m_50s__49m_0s.svg

In [15]:
!/scratch/enis/conda/envs/speechEnv/bin/python /home/enis/projects/nna/src/nna/slack_message.py -t "upload job ended" -m 'upload audio ended'
